## Miwok Entrants Database

### This notebook reads in the entrants table from miwok100k website and cross references ultrasignup results. 

### Probably won't be useful, since someone will eventually put the entrants list on UltraSignup ;-)

In [1]:
from bs4 import BeautifulSoup
import urllib2
import pandas as pd
import json
from IPython.display import HTML

response = urllib2.urlopen('http://miwok100k.com/site/lottery/')
html = response.read()
soup = BeautifulSoup(html)

entrant_table = soup.find("table", {"class" : "tableizer-table"})

# get the rows in the table (excluding header)
foo = entrant_table.findAll('tr')
header = foo[0]
entrants = foo[1:]

records = [] # store all of the records in this list
for row in entrants:
    col = row.findAll('td')
    firstname = col[0].string.strip()
    lastname = col[1].string.strip()
    city = col[2].string.strip()
    state = col[3].string.strip()
    record = '%s,%s,%s,%s' % (firstname, lastname, city, state) # store the record, comma separated
    records.append(record)

#split by comma and store in dataframe
records = [r.split(',') for r in records]

entrants = pd.DataFrame(records)
entrants.columns = ['First','Last','City','State']
# make first and last names have %20 instead of spaces
entrants['First'] = [r.replace(" ", "%20") for r in entrants['First']]
entrants['Last'] = [r.replace(" ", "%20") for r in entrants['Last']]

# replace 'jacob edman' with 'jake edman'
entrants.loc[(entrants['First']=='Jacob') & (entrants['Last']=='Edman'),'First'] = 'Jake'
entrants[entrants['Last']=='Edman']

# initialize gender, etc. columns
entrants['Gender'] = ''
entrants['AgeRank'] = 0
entrants['Rank'] = 0
entrants['Age'] = 0
entrants['NumRaces'] = 0
entrants['Link'] = ''
entrants['NumMatches'] = 0
entrants['Selected'] = 0 #index of runner selected (useful for de-duping)
#entrants.head()

/Users/stephanie/anaconda3/envs/snakes/lib/python2.7/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 162 of the file /Users/stephanie/anaconda3/envs/snakes/lib/python2.7/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [2]:
# This cell takes some time, it cross references the ultrasignup data
no_match = []
for i in range(len(entrants)):
    f = entrants.First[i]
    l = entrants.Last[i]
    try: 
        l.encode('ascii')
    except:
        print 'unicode error'
        no_match.append(i)
        entrants.loc[i,['Gender']] =  'None'
        entrants.loc[i,['Rank']] =  'None'
        entrants.loc[i,['AgeRank']] =  'None'
        continue
    try:
        r = urllib2.urlopen('https://ultrasignup.com/service/events.svc/history/' + f + '/' + l + '/')
        u = 'https://ultrasignup.com/results_participant.aspx?fname=' + f +'&lname=' + l
        u2 = '<a href="{0}">link</a>'.format(u)
        
        data = json.loads(r.read())
        entrants.loc[i,['NumMatches']] = len(data) #for resolving duplicate records
        
        # if num matches is >1, choose the runner with the larger # results
        if len(data) > 1:
            # make a list of the num races 
            races = [len(data[j]['Results']) for j in range(len(data))]
            #select index corresponding to largest races number
            ind = races.index(max(races))
        else:
            ind = 0 #there is only one runner, corresponding to index 0 in data
        
        entrants.loc[i,['Gender']] =  data[ind]['Gender']
        entrants.loc[i,['Rank']] =  data[ind]['Rank']
        entrants.loc[i,['AgeRank']] =  data[ind]['AgeRank']
        entrants.loc[i,['NumRaces']]= len(data[ind]['Results'])
        entrants.loc[i,['Age']]= data[ind]['Age']
        entrants.loc[i,['Link']]= u2 
        entrants.loc[i,['Selected']] = ind #keep track of which runner is chosen for de-duping
        
    except (IndexError, urllib2.HTTPError):
        no_match.append(i)
        entrants.loc[i,['Gender']] =  'None'
        entrants.loc[i,['Rank']] =  'None'
        entrants.loc[i,['AgeRank']] =  'None'
print 'there were ', len(no_match), 'entrants with no match in ultrasignup'

unicode error
unicode error
unicode error
there were  27 entrants with no match in ultrasignup


In [3]:
#manually deal with no-match people who you can find some info on or guess gender with confidence
entrants.loc[(entrants['First']=='Edouard') & (entrants['Last']=='Bulteau'),'Gender'] = 'M'
entrants.loc[(entrants['First']=='Rodrigo') & (entrants['Last']=='Cardoze'),'Gender'] = 'M'
entrants.loc[(entrants['First']=='Joseph') ,'Gender'] = 'M'
entrants.loc[(entrants['First']=='Jonathan'),'Gender'] = 'M'
entrants.loc[(entrants['First']=='Francisco'),'Gender'] = 'M'
entrants.loc[(entrants['First']=='William'),'Gender'] = 'M'
entrants.loc[(entrants['First']=='Jonathan'),'Gender'] = 'M'
entrants.loc[(entrants['First']=='Luis%20Francisco'),'Gender'] = 'M'
entrants.loc[(entrants['First']=='Maxwell'),'Gender'] = 'M'
entrants.loc[(entrants['First']=='Bill'),'Gender'] = 'M'
entrants.loc[(entrants['First']=='Kenneth'),'Gender'] = 'M'
entrants.loc[(entrants['First']=='Steven'),'Gender'] = 'M'
entrants.loc[(entrants['First']=='Jonathan'),'Gender'] = 'M'
entrants.loc[(entrants['First']=='David'),'Gender'] = 'M'
entrants.loc[(entrants['First']=='Phil'),'Gender'] = 'M'
entrants.loc[(entrants['First']=='Chris'),'Gender'] = 'M'
entrants.loc[(entrants['First']=='Colin'),'Gender'] = 'M'
entrants.loc[(entrants['First']=='Carlos'),'Gender'] = 'M'
entrants.loc[(entrants['First']=='Sean'),'Gender'] = 'M'
entrants.loc[(entrants['First']=='Adharanand'),'Gender'] = 'M'
entrants.loc[(entrants['First']=='Shawn'),'Gender'] = 'M'
entrants.loc[(entrants['First']=='Jiafan') & (entrants['Last']=='Yu'),'Gender'] = 'M'

# some females
entrants.loc[(entrants['First']=='Morgan') & (entrants['Last']=='Kennah'),'Gender'] = 'F'
entrants.loc[(entrants['First']=='Cam') & (entrants['Last']=='Karr'),'Gender'] = 'F'
entrants.loc[(entrants['First']=='Cam') & (entrants['Last']=='Karr'),'Rank'] = .6546
entrants.loc[(entrants['First']=='Cam') & (entrants['Last']=='Karr'),'AgeRank'] = .8444
entrants.loc[(entrants['First']=='Karolina ') & (entrants['Last']=='Lavagnino'),'Gender'] = 'F'
entrants.loc[(entrants['First']=='Karolina ') & (entrants['Last']=='Lavagnino'),'Rank'] = .7692
entrants.loc[(entrants['First']=='Karolina ') & (entrants['Last']=='Lavagnino'),'AgeRank'] = .8444
entrants.loc[(entrants['First']=='Alejandra') & (entrants['Last']=='Santos'),'Gender'] = 'F'
entrants.loc[(entrants['First']=='Ana%20Raquel') & (entrants['Last']=='Ulloa'),'Gender'] = 'F'

print 'There are', len(entrants[entrants['Gender']=='F']), 'women registered'
print 'Note there are still', len(entrants[entrants['Gender']=='None']), 'entrants of indeterminate gender'

There are 139 women registered
Note there are still 2 entrants of indeterminate gender


## Now Rank the Women According to UltraSignup "Rank"

In [4]:
pd.set_option('display.max_colwidth', -1)
#make separate female data frame :
F = entrants[(entrants['Gender']=='F') & (entrants['Rank']!='None')].sort_values(by='Rank', \
        ascending=0)
# include 'female rank' column and set it as index
F['FRank'] = list(range(len(F)))
F = F.set_index('FRank')
# list all the females, sort by rank, display as HTML so that link works
HTML(F[['First','Last','Rank','Age','NumRaces','Link']].head(30).to_html(escape=False))

,First,Last,Rank,Age,NumRaces,Link
FRank,,,,,,
0,Molly,Schmelzle,0.9901,36,11,link
1,Serena,Eley,0.9703,36,8,link
2,Christi,Richards,0.9626,29,26,link
3,Emily,Peterson,0.9514,30,17,link
4,Anna,Weisbrodt,0.949,35,31,link
5,Grace,Fisher,0.9132,36,25,link
6,Sarah,Mista,0.9123,38,10,link
7,Monica,DeVreese,0.8993,43,18,link
8,Lee,Conner,0.8984,43,55,link


## De-duplicating Scratch Work:

In [5]:
# check the duplicate female entries
HTML(entrants[(entrants['Gender']=='F') & (entrants['NumMatches']>1)][['First','Last', \
                                                        'Link','NumMatches']].to_html(escape=False))

,First,Last,Link,NumMatches
34,Flora,Bergstrom,link,2
51,Katie,Brooks,link,3
55,Patty,Bryant,link,2
61,Alison,Burke,link,2
66,Sandra,Campos,link,2
78,Lucia,Chacon,link,2
84,Luisa,Cheung,link,2
87,Amanda,Coffey,link,2
89,Jill,Cole,link,2
95,Jean,Coulter,link,2


In [6]:
# which runners have lots of ultrasignup people of same name (a couple are > 17!)
HTML(entrants[entrants['NumMatches']>17].to_html(escape=False))

,First,Last,City,State,Gender,AgeRank,Rank,Age,NumRaces,Link,NumMatches,Selected
289,Michael,Miller,Seattle,WA,M,0.9244,0.7881,54,155,link,19,0
392,Brian,Smith,Bluffton,SC,M,0.7386,0.671,49,32,link,18,4


In [7]:
# Scratch for testing one de-dup approach:

response = urllib2.urlopen('https://ultrasignup.com/service/events.svc/history/David/Cook/')
data = json.loads(response.read())
print len(data)

# thoughts for de-duping :
# 1) choose the runner with the larger # results
# 2) look into the states of the results to find a match for runner's home state.

# (1) is implemented above as so:

# make a dictionary of index:num results for de-duping
races = [len(data[i]['Results']) for i in range(len(data))]
print races
#select index corresponding to largest races number
ind = races.index(max(races))
print 'max num races is ', races[ind], ' at index ', ind

9
[17, 16, 2, 18, 3, 1, 2, 4, 2]
max num races is  18  at index  3


In [8]:
#Scratch for querying a single person's ultrasignup data
response = urllib2.urlopen('https://ultrasignup.com/service/events.svc/history/jake/edman/') #lowercase works!
data = json.loads(response.read())
print "Rank: ", data[0]['Rank']
print "AgeRank: ", data[0]['AgeRank']
print len(data[0]['Results'])
#data[0]

response = urllib2.urlopen('https://ultrasignup.com/service/events.svc/history/Serena/Eley/')
data = json.loads(response.read())
print "Rank: ", data[0]['Rank']
print "AgeRank: ", data[0]['AgeRank']
print len(data[0]['Results'])
#data[0]

Rank:  0.8155
AgeRank:  0.8531
4
Rank:  0.9703
AgeRank:  0.9746
8
